In [1]:
import duckdb
from pathlib import Path

DATA_PROCESSED_DIR = Path("data/processed").resolve()
DATA_PROCESSED_DIR


PosixPath('/Users/joeldiamond/Projects/imdb-data-engineering/data/processed')

In [2]:
# This will create imdb.duckdb in your project root if it doesn't already exist
con = duckdb.connect("imdb.duckdb")

con


In [3]:
list(DATA_PROCESSED_DIR.glob("*.parquet"))


[PosixPath('/Users/joeldiamond/Projects/imdb-data-engineering/data/processed/name_basics.parquet'),
 PosixPath('/Users/joeldiamond/Projects/imdb-data-engineering/data/processed/title_basics.parquet'),
 PosixPath('/Users/joeldiamond/Projects/imdb-data-engineering/data/processed/title_ratings.parquet'),
 PosixPath('/Users/joeldiamond/Projects/imdb-data-engineering/data/processed/title_crew.parquet')]

In [5]:
# title_basics
path = DATA_PROCESSED_DIR / "title_basics.parquet"
con.execute(f"""
    CREATE OR REPLACE VIEW title_basics AS
    SELECT * FROM read_parquet('{path}')
""")

# title_ratings
path = DATA_PROCESSED_DIR / "title_ratings.parquet"
con.execute(f"""
    CREATE OR REPLACE VIEW title_ratings AS
    SELECT * FROM read_parquet('{path}')
""")

# name_basics
path = DATA_PROCESSED_DIR / "name_basics.parquet"
con.execute(f"""
    CREATE OR REPLACE VIEW name_basics AS
    SELECT * FROM read_parquet('{path}')
""")

# title_crew
path = DATA_PROCESSED_DIR / "title_crew.parquet"
con.execute(f"""
    CREATE OR REPLACE VIEW title_crew AS
    SELECT * FROM read_parquet('{path}')
""")


In [6]:
con.execute("SHOW TABLES").df()


,name
0,name_basics
1,title_basics
2,title_crew
3,title_ratings


In [8]:
con.execute("DESCRIBE title_basics").df()
con.execute("DESCRIBE title_ratings").df()
con.execute("DESCRIBE name_basics").df()
con.execute("DESCRIBE title_crew").df()


,column_name,column_type,null,key,default,extra
0,tconst,VARCHAR,YES,None,None,None
1,directors,VARCHAR[],YES,None,None,None
2,writers,VARCHAR[],YES,None,None,None


In [9]:
con.execute("SELECT COUNT(*) FROM title_basics").df()
con.execute("SELECT COUNT(*) FROM title_ratings").df()
con.execute("SELECT COUNT(*) FROM name_basics").df()
con.execute("SELECT COUNT(*) FROM title_crew").df()


,count_star()
0,12091590


In [10]:
con.execute("""
    SELECT
        b.tconst,
        b.primaryTitle,
        b.startYear,
        b.genres,
        r.averageRating,
        r.numVotes
    FROM title_basics AS b
    JOIN title_ratings AS r
      ON b.tconst = r.tconst
    ORDER BY r.numVotes DESC
    LIMIT 10
""").df()


,tconst,primaryTitle,startYear,genres,averageRating,numVotes
0,tt0111161,The Shawshank Redemption,1994.0,[Drama],9.3,3125414
1,tt0468569,The Dark Knight,2008.0,"[Action, Crime, Drama]",9.1,3100928
2,tt1375666,Inception,2010.0,"[Action, Adventure, Sci-Fi]",8.8,2755768
3,tt0137523,Fight Club,1999.0,"[Crime, Drama, Thriller]",8.8,2540297
4,tt0944947,Game of Thrones,2011.0,"[Action, Adventure, Drama]",9.2,2504192
5,tt0109830,Forrest Gump,1994.0,"[Drama, Romance]",8.8,2442298
6,tt0816692,Interstellar,2014.0,"[Adventure, Drama, Sci-Fi]",8.7,2435960
7,tt0903747,Breaking Bad,2008.0,"[Crime, Drama, Thriller]",9.5,2431739
8,tt0110912,Pulp Fiction,1994.0,"[Crime, Drama]",8.8,2387535
9,tt0133093,The Matrix,1999.0,"[Action, Sci-Fi]",8.7,2205165


In [ ]:
con.execute("""
    SELECT
        b.tconst,
        b.primaryTitle,
        b.startYear,
        b.genres,
        r.averageRating,
        r.numVotes
    FROM title_basics AS b
    JOIN title_ratings AS r
      ON b.tconst = r.tconst
    WHERE b.titleType = 'movie'
      AND r.numVotes >= 50000
    ORDER BY r.averageRating DESC, r.numVotes DESC
    LIMIT 20;
""").df()


In [ ]:

con.execute("""
    SELECT
        (startYear/10)*10 AS decade,
        COUNT(*) AS n_titles
    FROM title_basics
    WHERE titleType = 'movie'
      AND startYear IS NOT NULL
      AND startYear >= 1900
    GROUP BY decade
    ORDER BY decade;
""").df()


In [ ]:
con.execute("""
    WITH exploded AS (
        SELECT
            b.tconst,
            b.primaryTitle,
            b.startYear,
            UNNEST(string_split(b.genres, ',')) AS genre,
            r.averageRating,
            r.numVotes
        FROM title_basics AS b
        JOIN title_ratings AS r ON b.tconst = r.tconst
        WHERE b.titleType = 'movie'
          AND b.genres IS NOT NULL
          AND r.numVotes >= 5000
    )
    SELECT
        genre,
        COUNT(*) AS n_titles,
        AVG(averageRating) AS avg_rating,
        SUM(numVotes) AS total_votes
    FROM exploded
    GROUP BY genre
    HAVING COUNT(*) >= 100
    ORDER BY avg_rating DESC
    LIMIT 20;
""").df()


In [ ]:
con.execute("""
    WITH exploded AS (
        SELECT
            b.tconst,
            b.primaryTitle,
            b.startYear,
            UNNEST(string_split(b.genres, ',')) AS genre,
            r.numVotes
        FROM title_basics AS b
        JOIN title_ratings AS r ON b.tconst = r.tconst
        WHERE b.titleType = 'movie'
          AND b.genres IS NOT NULL
          AND b.startYear >= 2000
          AND r.numVotes >= 10000
    )
    SELECT
        genre,
        COUNT(*) AS n_titles,
        SUM(numVotes) AS total_votes
    FROM exploded
    GROUP BY genre
    ORDER BY total_votes DESC
    LIMIT 10;
""").df()


In [ ]:
con.execute("""
    WITH exploded_directors AS (
        SELECT
            c.tconst,
            UNNEST(string_split(c.directors, ',')) AS nconst
        FROM title_crew AS c
        WHERE c.directors IS NOT NULL
    ),
    joined AS (
        SELECT
            e.tconst,
            e.nconst,
            n.primaryName AS director_name,
            r.averageRating,
            r.numVotes
        FROM exploded_directors e
        JOIN name_basics n ON e.nconst = n.nconst
        JOIN title_ratings r ON r.tconst = e.tconst
    )
    SELECT
        director_name,
        COUNT(*) AS films_directed,
        AVG(averageRating) AS avg_rating,
        SUM(numVotes) AS total_votes
    FROM joined
    GROUP BY director_name
    HAVING COUNT(*) >= 5
    ORDER BY avg_rating DESC
    LIMIT 20;
""").df()


In [ ]:
con.execute("""
    SELECT
        CASE
            WHEN numVotes < 1000 THEN '<1k'
            WHEN numVotes < 10000 THEN '1k-10k'
            WHEN numVotes < 50000 THEN '10k-50k'
            WHEN numVotes < 100000 THEN '50k-100k'
            ELSE '100k+'
        END AS vote_bin,
        COUNT(*) AS n_titles,
        AVG(averageRating) AS avg_rating
    FROM title_ratings
    GROUP BY vote_bin
    ORDER BY n_titles DESC;
""").df()


# Modeling Notes (Week 4)

## Candidate Dimensions
- **Title Dimension**: tconst, titleType, primaryTitle, originalTitle, startYear, runtimeMinutes, genres
- **Person Dimension**: nconst, primaryName, birthYear, deathYear, professions
- **Genre Dimension**: derived from splitting genres
- **Date/Year Dimension**: startYear, decade

## Candidate Fact Tables
- **Ratings Fact**:
  - grain: title (tconst)
  - measures: averageRating, numVotes

## Bridge Tables
- **Title–Director** (many-to-many)
- **Title–Writer**

## Notes
- Genres require explosion → classic multi-valued attribute → fits into a bridge or dimension table.
- Ratings behave like a snapshot fact table.
